In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
np.random.seed(0)

In [4]:
df_orig = pd.read_csv('../wf_df_raw.csv')

In [5]:
df = df_orig.copy()

In [7]:
BASE_DIR = os.path.dirname(os.getcwd())

RAW_IMGS_DIR = os.path.join(BASE_DIR, 'raw_images')

IMAGES_DIR = os.path.join(BASE_DIR, 'Images')
TRAINING_DIR = os.path.join(IMAGES_DIR, 'Training')
VALIDATION_DIR = os.path.join(IMAGES_DIR, 'Validation')

TRAIN_ROLEX_DIR = os.path.join(TRAINING_DIR, 'Rolex')
TRAIN_NOT_ROLEX_DIR = os.path.join(TRAINING_DIR, 'Not_Rolex')
VAL_ROLEX_DIR = os.path.join(VALIDATION_DIR, 'Rolex')
VAL_NOT_ROLEX_DIR = os.path.join(VALIDATION_DIR, 'Not_Rolex')

In [23]:
try:
    shutil.rmtree(IMAGES_DIR)
except: pass

os.mkdir(IMAGES_DIR)
os.mkdir(TRAINING_DIR)
os.mkdir(VALIDATION_DIR)

os.mkdir(TRAIN_ROLEX_DIR)
os.mkdir(TRAIN_NOT_ROLEX_DIR)
os.mkdir(VAL_ROLEX_DIR)
os.mkdir(VAL_NOT_ROLEX_DIR)

In [24]:
df = df.dropna(subset=['img_name'])

In [25]:
for index, row in df.iterrows():
    
    CURR_IMG_DIR = os.path.join(RAW_IMGS_DIR, row['img_name'])
    
    random_number = np.random.randint(1,101)
    
    if random_number <= 75 and row['brand'] == 'Rolex':
        shutil.copy(CURR_IMG_DIR, TRAIN_ROLEX_DIR)
    
    elif random_number <= 75 and row['brand'] != 'Rolex':
        shutil.copy(CURR_IMG_DIR, TRAIN_NOT_ROLEX_DIR)
    
    elif random_number > 75 and row['brand'] == 'Rolex':
        shutil.copy(CURR_IMG_DIR, VAL_ROLEX_DIR)
    
    elif random_number > 75 and row['brand'] != 'Rolex':
        shutil.copy(CURR_IMG_DIR, VAL_NOT_ROLEX_DIR)

In [26]:
print(f"Train Rolex num: {len(os.listdir(TRAIN_ROLEX_DIR))}")
print(f"Train Not Rolex num: {len(os.listdir(TRAIN_NOT_ROLEX_DIR))}")
print(f"Val Rolex num: {len(os.listdir(VAL_ROLEX_DIR))}")
print(f"Val Not Rolex num: {len(os.listdir(VAL_NOT_ROLEX_DIR))}")
print()
print(round(100 * (1 - len(os.listdir(TRAIN_ROLEX_DIR)) / len(os.listdir(TRAIN_NOT_ROLEX_DIR))), 2))
print(round(100 * (1 - len(os.listdir(VAL_ROLEX_DIR)) / len(os.listdir(VAL_NOT_ROLEX_DIR))), 2))

Train Rolex num: 603
Train Not Rolex num: 2743
Val Rolex num: 191
Val Not Rolex num: 931

78.02
79.48


In [27]:
train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(300, 300),
    class_mode='binary'
)

Found 3346 images belonging to 2 classes.


In [31]:
validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(300, 300),
    class_mode='binary'
)

Found 1122 images belonging to 2 classes.


In [32]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                           input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [33]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
              metrics=['accuracy'])

In [37]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=5
)

Epoch 1/5
105/105 [==============================] - 101s 968ms/step - loss: 0.1828 - accuracy: 0.9325 - val_loss: 0.1396 - val_accuracy: 0.9474
Epoch 2/5
105/105 [==============================] - 99s 948ms/step - loss: 0.1157 - accuracy: 0.9597 - val_loss: 0.0989 - val_accuracy: 0.9679
Epoch 3/5
105/105 [==============================] - 99s 949ms/step - loss: 0.0685 - accuracy: 0.9767 - val_loss: 0.1007 - val_accuracy: 0.9635
Epoch 4/5
105/105 [==============================] - 100s 950ms/step - loss: 0.0773 - accuracy: 0.9839 - val_loss: 0.0858 - val_accuracy: 0.9670
Epoch 5/5
105/105 [==============================] - 101s 964ms/step - loss: 0.0288 - accuracy: 0.9913 - val_loss: 0.0239 - val_accuracy: 0.9893


In [38]:
from PIL import Image

folder_path = TRAINING_DIR
extensions = ['.jpg']
for fldr in os.listdir(folder_path):
    sub_folder_path = os.path.join(folder_path, fldr)
    for filee in os.listdir(sub_folder_path):
        file_path = os.path.join(sub_folder_path, filee)
        print('** Path: {}  **'.format(file_path), end="\r", flush=True)
        im = Image.open(file_path)
        rgb_im = im.convert('RGB')
        if filee.split('.')[1] not in extensions:
            extensions.append(filee.split('.')[1])